In [1]:
import requests
from bs4 import BeautifulSoup 
import csv
from itertools import zip_longest 
import pandas as pd 
from tqdm import tqdm
import time 

In [2]:
Company_caegory_links=[]
Company_city_links=[]
company_name=[]
company_category=[]
company_city=[]
company_activity=[]
company_phone=[]
company_link=[]

In [3]:
main_link ="https://www.egycompanies.com/"

In [4]:
def find_categorys_city(main_link):
  Company_caegory_links=[]
  # Company_city_links=[]
  request = requests.get(main_link)
  src =request.content
  soup =BeautifulSoup(src ,"lxml")
  Company_city_soup=soup.find("div",{"id":"demozones"}).find("section").find("div").find_all("p")
  Company_category_soup=soup.find("div",{"id":"demo"}).find("section").find("div").find_all("p")

  for i in range(len(Company_city_soup)):
      Company_caegory_links.append(Company_city_soup[i].find('a')['href'])
  for i in range(len(Company_category_soup)):
      Company_caegory_links.append(Company_category_soup[i].find('a')['href'])
  return Company_caegory_links
Company_caegory_links=find_categorys_city(main_link)


In [6]:
len(Company_caegory_links)

32

In [7]:
# collect data with category partitions 
for t in tqdm(range(0,len(Company_caegory_links))):
  request = requests.get(Company_caegory_links[t])
  src =request.content
  soup =BeautifulSoup(src ,"lxml")
  try:
    citys_length =int(soup.find("li",{"id":"news_next"}).find_all("a")[1]["href"].split("=")[1])
  except:
    citys_length=1
  for j in range(1,citys_length+1):
    request = requests.get(Company_caegory_links[t]+"?page="+str(j))
    src =request.content
    soup =BeautifulSoup(src ,"lxml")
    names=soup.find_all("h1",{"class":"f-listings-item__title"})
    phones=soup.find_all("address",{"class":"f-listings-item__address"})
    category=soup.find_all("ul",{"class":"wil-icon-list"})
    for i in range(len(names)):
        company_name.append(names[i].text)
        company_link.append(names[i].find("a")["href"])
        company_phone.append(phones[i].text[1:-1].split("\n")[0])
        try:
          company_activity.append(phones[i].text[1:-1].split("\n")[1])
        except:
          company_activity.append(" ")
        company_category.append(category[0].find("li").find("span").find("h2").text)

100%|██████████| 32/32 [37:11<00:00, 69.74s/it]


In [8]:
len(company_activity)

16564

In [ ]:
request = requests.get("https://www.egycompanies.com/SearchR/1/Page/1/Zone/0/Industry/21/%D8%A7%D9%84%D9%88%D8%B1%D9%82-%D9%88%D8%A7%D9%84%D9%83%D8%B1%D8%AA%D9%88%D9%86")
src =request.content
soup =BeautifulSoup(src ,"lxml")
names=soup.find_all("h1",{"class":"f-listings-item__title"})
phones=soup.find_all("address",{"class":"f-listings-item__address"})
category=soup.find_all("ul",{"class":"wil-icon-list"})
phones[].text[1:-1].split("\n")[1]

' النشاط:   لوازم محطات معالجة مياه'

In [ ]:
category[0].find("li").find("span").find("h2").text#[1:].split("\n")[0][:-3].strip()

'معالجة المياه والفلاتر'

In [9]:
df=pd.DataFrame({
    "company_name":company_name,
    "company_phone":company_phone,
    "company_category":company_category,
    # "company_city":company_city,
    "company_activity":company_activity,
    "company_link":company_link
})
df


,company_name,company_phone,company_category,company_activity,company_link
0,الجمعية التعاونية للأدوية | أدوية بشرية,هاتف الإدارة: 0504673762,أدوية وتجميل ومستلزمات,النشاط: أدوية بشرية,https://www.egycompanies.com/SearchR/1/Page/1/...
1,الشركة الحديثة للتجارة | أدوية بشرية - مستلزما...,هاتف الإدارة: 0482221294,أدوية وتجميل ومستلزمات,النشاط: أدوية بشرية - مستلزمات طبية,https://www.egycompanies.com/SearchR/1/Page/1/...
2,الشركة العربية للصناعات الدقيقة - أبيك | أجهزة...,هاتف الشركة: 0244893437,أدوية وتجميل ومستلزمات,النشاط: أجهزة أشعة سينية,https://www.egycompanies.com/SearchR/1/Page/1/...
3,الشركة العربية لمستحضرات التجميل | مستحضرات تج...,هاتف الإدارة: 0237829309,أدوية وتجميل ومستلزمات,النشاط: مستحضرات تجميل - كريمات بشرة - كريم...,https://www.egycompanies.com/SearchR/1/Page/1/...
4,الشركة العلمية للمعدات الطبية | معدات طبية,هاتف الإدارة: 0222561307,أدوية وتجميل ومستلزمات,النشاط: معدات طبية,https://www.egycompanies.com/SearchR/1/Page/1/...
...,...,...,...,...,...
16559,مجموعة شركات المنار | وحدات معالجة مياه,هاتف الإدارة: 0224052387,معالجة المياه والفلاتر,النشاط: وحدات معالجة مياه,https://www.egycompanies.com/SearchR/1/Page/1/...
16560,معرض التوأم لتجارة المولدات ومعدات البناء | طل...,الموبيل: 01225197779,معالجة المياه والفلاتر,النشاط: طلمبات مياه - مولدات كهرباء,https://www.egycompanies.com/SearchR/1/Page/1/...
16561,معرض مصر الدولي لأحواض السباحة وتكنولوجيا المي...,هاتف الإدارة: 0233842040,معالجة المياه والفلاتر,النشاط: معدات حمامات سباحة,https://www.egycompanies.com/SearchR/1/Page/1/...
16562,مكتب القاهرة الفني الهندسي | حفر آبار مياه - ط...,هاتف الإدارة: 0225905454,معالجة المياه والفلاتر,النشاط: حفر آبار مياه - طلمبات مياه,https://www.egycompanies.com/SearchR/1/Page/1/...


In [10]:
df.to_excel(" دليل الشركات المصرية نسخة القطاعات التجارية .xlsx")